```bash
# ================================
# Kali Linux — Command Cheat Sheet
# ================================

# 1) System updates & cleanup
sudo apt update && sudo apt full-upgrade -y
sudo apt autoremove --purge -y
sudo apt clean

# 2) Users, groups, permissions (least privilege)
sudo adduser <username>
sudo usermod -aG sudo <username>
sudo passwd -l root                       # (opz.) blocca login root locale
sudo chown <user>:<group> <path>          # cambia proprietario
sudo chmod <perm> <path>                  # cambia permessi (es. 600, 700, 750)
chmod -R 700 ~/.ssh && chmod 600 ~/.ssh/* # permessi sicuri per SSH keys

# 3) Two-Factor Authentication (Google Authenticator)
sudo apt install -y libpam-google-authenticator
google-authenticator                      # setup interattivo per l’utente corrente
echo "auth required pam_google_authenticator.so nullok" | sudo tee -a /etc/pam.d/sshd
sudo sed -i 's/^#\?ChallengeResponseAuthentication.*/ChallengeResponseAuthentication yes/' /etc/ssh/sshd_config
sudo systemctl restart ssh

# 4) Service hardening (disabilita ciò che non serve)
systemctl list-unit-files --type=service --state=enabled
sudo systemctl disable --now <service_name>
sudo systemctl mask <service_name>        # impedisce avvio anche manuale

# 5) Firewall (UFW)
sudo apt install -y ufw
sudo ufw default deny incoming
sudo ufw default allow outgoing
sudo ufw allow 22/tcp                     # o la porta SSH che userai (vedi sezione SSH)
# Esempi più restrittivi:
# sudo ufw allow from <IP>/32 to any port 2222 proto tcp
sudo ufw enable
sudo ufw status verbose

# 6) SSH hardening
sudo apt install -y openssh-server
sudo sed -i 's/^#\?PermitRootLogin.*/PermitRootLogin no/' /etc/ssh/sshd_config
sudo sed -i 's/^#\?PasswordAuthentication.*/PasswordAuthentication no/' /etc/ssh/sshd_config
sudo sed -i 's/^#\?Port .*/Port 2222/' /etc/ssh/sshd_config   # scegli una porta non standard
# (opz.) limita utenti/ip:
# echo -e "AllowUsers <user1> <user2>\n" | sudo tee -a /etc/ssh/sshd_config
sudo systemctl restart ssh

# 7) VPN (OpenVPN / WireGuard)
sudo apt install -y openvpn
sudo openvpn --config /path/to/profile.ovpn
sudo apt install -y wireguard
sudo wg-quick up wg0                       # richiede /etc/wireguard/wg0.conf

# 8) Backup (rsync + Timeshift)
# Esempio rsync (backup incrementale con preservazione ACL/xattr):
sudo rsync -aAXHv --delete --numeric-ids --info=progress2 \
  /etc /home /var/backups /usr/local \
  /mnt/backup/kali-$(date +%F)/
# Timeshift (snapshot di sistema — utile su installazioni desktop):
sudo apt install -y timeshift
sudo timeshift --create --comments "pre-change" --tags D

# 9) Cifratura dati
sudo apt install -y veracrypt
# GnuPG (cifratura simmetrica/asimmetrica):
gpg --gen-key
gpg -c <file>                              # cifratura simmetrica -> <file>.gpg
gpg -e -r "<Your GPG Name or Email>" <file>
gpg -d <file>.gpg > <file>.dec

# 10) IDPS (Intrusion Detection & Prevention)
sudo apt install -y snort
sudo snort -T -c /etc/snort/snort.conf     # test configurazione
sudo apt install -y suricata
sudo suricata -T -c /etc/suricata/suricata.yaml
sudo systemctl enable --now suricata
# (opz.) OSSEC / Wazuh (se disponibile nei repo configurati):
# sudo apt install -y ossec-hids
# sudo systemctl enable --now ossec

# 11) Monitoring rapido (log SSH)
sudo journalctl -u ssh --since "today"
sudo tail -n 100 /var/log/auth.log

# 12) Pianificazione (cron esempio per backup settimanale con rsync)
# crontab -e
# 0 3 * * 1 sudo rsync -aAXHv --delete /etc /home /var/backups /usr/local /mnt/backup/kali-$(date +\%F)/
```
